#### 1. Create and Test Custom Environment with OpenAi Gym

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [8]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take: down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60  

    def step(self, action):
        # Apply action
        self.state += action - 1
        # Reduce shower_length by 1 second
        self.shower_length -= 1

        # Calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False

        # Apply temperature noise
        self.state += random.randint(-1, 1)
        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement visualization
        pass

    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60
        return self.state        


In [10]:
env = ShowerEnv()

In [19]:
env.action_space.sample()

1

In [20]:
env.observation_space.sample()

array([9.680166], dtype=float32)

In [22]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        # env.render()
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode: {episode} Score:{score}')    


Episode: 1 Score:-60
Episode: 2 Score:-22
Episode: 3 Score:-44
Episode: 4 Score:-54
Episode: 5 Score:-6
Episode: 6 Score:-54
Episode: 7 Score:-50
Episode: 8 Score:-24
Episode: 9 Score:-42
Episode: 10 Score:-18


#### 2. Create a Deep Learning Model with Keras


In [23]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [26]:
states = env.observation_space.shape
actions = env.action_space.n

In [27]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [37]:
model = build_model(states, actions)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 24)                48        
                                                                 
 dense_10 (Dense)            (None, 24)                600       
                                                                 
 dense_11 (Dense)            (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


#### 3. Build Agent with Keras-RL

In [38]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [39]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, policy=policy, memory=memory,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [58]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 9:28 - reward: 1.0000

/home/senai/anaconda3/envs/gym/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/home/senai/anaconda3/envs/gym/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 47s 5ms/step - reward: -0.5472
166 episodes - episode_reward: -32.843 [-60.000, 28.000] - loss: 3.169 - mae: 12.989 - mean_q: -18.768

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: -0.4170
167 episodes - episode_reward: -25.066 [-60.000, 30.000] - loss: 2.553 - mae: 11.208 - mean_q: -16.093

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: -0.4062
167 episodes - episode_reward: -24.347 [-60.000, 26.000] - loss: 1.734 - mae: 8.814 - mean_q: -12.479

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -0.2820
166 episodes - episode_reward: -16.916 [-56.000, 32.000] - loss: 1.053 - mae: 5.769 - mean_q: -7.832

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 50s 5ms/step - reward: -0.0882
done, took 240.915 seconds
